In [ ]:
from darts.models import TiDEModel
from darts.dataprocessing.transformers.scaler import Scaler
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from torch.optim import lr_scheduler

In [ ]:
import sys
import os

# Go up two levels from notebook (Training/MLR) to project root
project_root = os.path.abspath(os.path.join(os.getcwd(), "../.."))
sys.path.append(project_root)

print("Project root added to sys.path:", project_root)
# Ensure the model save directory exists
model_save_path = os.path.join('.')
os.makedirs(model_save_path, exist_ok=True)  # Creates directory if it doesn't exist

Data loading

In [ ]:
import pandas as pd
from Training.Helper.dataPreprocessing import TRAIN_DATA_PATH_1990S, get_untransformed_exog
date_col = 'observation_date'

# Load and format training data (only using PCEPI)
train_df = pd.read_csv(TRAIN_DATA_PATH_1990S)
train_df = get_untransformed_exog(train_df)
# Convert the date column to datetime format
train_df[date_col] = pd.to_datetime(train_df[date_col], format='%m/%Y')

# Set the date column as the index
train_df.set_index(date_col, inplace=True)

In [ ]:
from darts import TimeSeries
target_series = TimeSeries.from_series(train_df['fred_PCEPI'])

# Extract the exogenous variables (all columns except 'fred_PCEPI')
exogenous_variables = train_df.drop(columns=['fred_PCEPI'])
exogenous_series = TimeSeries.from_dataframe(exogenous_variables)

Basic model

In [ ]:
model = TiDEModel(input_chunk_length=24, output_chunk_length=6)

# Fit the model with the target series and exogenous variables
model.fit(series=target_series,past_covariates=exogenous_series)

In [ ]:
pred = model.predict(6)
pred.values()

This is terrible 

Split validation and training then scale

In [ ]:
from Training.Helper.dataPreprocessing import TRAIN_DATA_SPLIT
train_target, val_target = target_series.split_after(TRAIN_DATA_SPLIT)
train_exo, val_exo = exogenous_series.split_after(TRAIN_DATA_SPLIT)

In [ ]:
targetScaler = Scaler()  # default uses sklearn's MinMaxScaler
scaled_train_target = targetScaler.fit_transform(train_target)
scaled_val_target = targetScaler.transform(val_target)

exoScaler = Scaler() 
scaled_train_exo = exoScaler.fit_transform(train_exo)
scaled_val_exo = exoScaler.transform(val_exo)

In [ ]:
scaled_train_target.plot(label="train")
scaled_val_target.plot(label="val")

Basic model

In [ ]:
OUT_LENGTH = 12

early_stopper = EarlyStopping(
    monitor='val_loss',
    patience=10,
    min_delta=1e-3,
    mode='min'
)
lr_scheduler_kwargs = {
    "gamma": 0.999,
}
model = TiDEModel(
    input_chunk_length=48,
    output_chunk_length=OUT_LENGTH,
    pl_trainer_kwargs={"callbacks": [early_stopper]},
    optimizer_kwargs={"lr": 1e-3},
    lr_scheduler_cls= lr_scheduler.ExponentialLR,
    lr_scheduler_kwargs = {"gamma": 0.999},
    use_reversible_instance_norm = True
)

In [ ]:
import copy

model.fit(series=scaled_train_target,past_covariates=scaled_train_exo,val_series=scaled_val_target,val_past_covariates=scaled_val_exo,verbose=True)
# Make a copy of the model for potential later use
original_model = copy.deepcopy(model)

In [ ]:
# prediction_size must be <= OUT_LENGTH
prediction_size = OUT_LENGTH
predictions = model.predict(prediction_size, verbose=False)
transformed_predictions = targetScaler.inverse_transform(predictions)

In [ ]:
from darts.metrics import mse
from Evaluation.Helper.evaluation_helpers import calc_metrics_arrays

train_target[-prediction_size:].plot(label='train')
val_target[:prediction_size].plot(label='val')
transformed_predictions.plot(label='predictions')
original_model_metrics = calc_metrics_arrays(val_target[:prediction_size].values(), transformed_predictions.values(), model_names=['TiDE'])
print('Metrics for first model without hyperparameter optimisation:')
display(original_model_metrics)

The discontinuity in the above plot between the ground-truth training and validation sets looks incorrect, but that is simply because a natural slight downturn occurred between the two months in question at the end of the training period and the beginning of the validation period respectively.

Not that good so optuna test

In [ ]:
# This is currently not used, but could be switched to
split_date = pd.Timestamp('2022-12')
op_train , op_val = target_series.split_after(split_date)
op_train_exo , op_val_exo = exogenous_series.split_after(split_date)

In [ ]:
import optuna
from darts.metrics import mse
from Training.Helper.PyTorchModular import optuna_trial_get_kwargs

model_search_space = {
    'input_chunk_length': (int, (24, 60)),
    'num_encoder_layers': (int, (1, 3)),
    'num_decoder_layers': (int, (1, 3)),
    'hidden_size': (int, (64, 512)),
    'dropout': (float, (0.1, 0.5)),
    'optimizer_kwargs': {"lr": (float, (1e-4, 1e-2))},
    'lr_scheduler_kwargs': {"gamma": (float, (0.9, 1.0))},
    'use_reversible_instance_norm': ('categorical', [True, False]),
}

# Controlling input chunk length for now to decrease the size of the search space
model_invariates = {
    #'input_chunk_length': 48,
    'output_chunk_length': 12,
    'lr_scheduler_cls': lr_scheduler.ExponentialLR,
    'pl_trainer_kwargs': {"callbacks": [early_stopper]}
}

def objective(trial):

    model_kwargs = optuna_trial_get_kwargs(trial, model_search_space)

    # Initialize the TiDEModel with suggested hyperparameters
    model = TiDEModel(**model_kwargs, **model_invariates)

    # Fit the model
    model.fit(series = scaled_train_target,
              past_covariates = scaled_train_exo,
              val_series = scaled_val_target,
              val_past_covariates = scaled_val_exo,
              epochs=1000,
              verbose = False)

    # Evaluate the model
    # (this is an alternative option for evaluation, where the model must predict the final prediction_size elements of the validation data having been given all other validation data;
    #  if switching to this method, ensure that final prediction is performed with the same setup (this is currently done just by predicting the next n values))
    #scaled_val_predictions = model.predict(n=prediction_size,series=scaled_val_target[:-prediction_size],past_covariates=scaled_val_exo[:-prediction_size], verbose=False)]
    #val_predictions = targetScaler.inverse_transform(scaled_val_predictions, verbose=False)
    #error = mse(val_target[-prediction_size:], val_predictions, verbose=False)

    # Raw output is scaled, so inverse transform to become comparable with validation set
    scaled_val_predictions = model.predict(n=prediction_size, verbose=False)
    val_predictions = targetScaler.inverse_transform(scaled_val_predictions, verbose=False)
    # Only uses the first prediction_size values of val_target, since this is the size of the prediction made by the model
    error = mse(val_target[:prediction_size], val_predictions, verbose=False)
    return error


In [ ]:
# Create an Optuna study and optimize
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=50)

In [ ]:
from Training.Helper.PyTorchModular import MAX_DEPTH

def reformat_best_params(best_params, model_search_space, cur_depth=0):
    if cur_depth > MAX_DEPTH: raise RecursionError(f'Cannot exceed recursion depth of {MAX_DEPTH}')
    reformatted = {}
    for key in model_search_space:
        if key in best_params: reformatted[key] = best_params[key]
        else: reformatted[key] = reformat_best_params(best_params, model_search_space[key], cur_depth=cur_depth+1)
    return reformatted

In [ ]:
# Retrieve the best hyperparameters
best_params = study.best_params
# Format parameters returned by study into the same style as the search space definition (can be passed straight into model as kwargs)
best_params = reformat_best_params(best_params, model_search_space)
print('Best hyperparameters:')
display(best_params)

Best params made into a model here

In [ ]:
# Initialize the TiDEModel with suggested hyperparameters
best_model = TiDEModel(**best_params, **model_invariates)

In [ ]:
# Fossilised suggested hyperparameters
    # Define early stopping callback
#early_stopper = EarlyStopping(
#    monitor='val_loss',
#    patience=10,
#    min_delta=1e-3,
#    mode='min'
#)
#model = TiDEModel(
#    input_chunk_length=29,
#    output_chunk_length=12,
#    num_encoder_layers=3,
#    num_decoder_layers=2,
#    hidden_size=443,
#    dropout= 0.19411763114257125,
#    optimizer_kwargs={"lr": 0.00014544898516544107},
#    lr_scheduler_cls=torch.optim.lr_scheduler.ExponentialLR,
#    lr_scheduler_kwargs={"gamma": 0.9645025339005199},
#    pl_trainer_kwargs={"callbacks": [early_stopper]},
#)

In [ ]:
# Fit the model
best_model.fit(series=scaled_train_target,past_covariates=scaled_train_exo, val_series=scaled_val_target, val_past_covariates=scaled_val_exo)

In [ ]:
# Predict over the test horizon
TEST_HORIZON = 12
val_predictions = best_model.predict(n=TEST_HORIZON)
transformed_predictions = targetScaler.inverse_transform(val_predictions)

In [ ]:
from Evaluation.Helper.evaluation_helpers import display_results

actuals = val_target[:TEST_HORIZON].values()
preds = transformed_predictions.values()

display_results(actuals, preds, val_target[:TEST_HORIZON].time_index, 'TiDE')

In [ ]:
# Evaluate the model
print(f'Optuna trained model metrics on validation set:')
best_model_metrics = calc_metrics_arrays(actuals, preds, model_names=['TiDE'])
display(best_model_metrics)

In [ ]:
display(original_model_metrics, best_model_metrics)

In [ ]:
# Find the model from the that has the best RMSE and take that as the best
if original_model_metrics.values.flatten()[0] < best_model_metrics.values.flatten()[0]:
    best_model = original_model

In [ ]:
best_model.save("tide.pkl")

Go from here if wanting to load model

In [ ]:
from darts.models import TiDEModel

# Load the model from the file
model = TiDEModel.load("tide.pkl")

In [ ]:
# Finally, predict 12 months into the future from the end of the training dataset
scaled_total = targetScaler.transform(target_series)
scaled_total_exo = exoScaler.transform(exogenous_series)

pred = model.predict(12,series=scaled_total,past_covariates=scaled_total_exo)
finalout = targetScaler.inverse_transform(pred)
print(f'Final predictions:\n{finalout.to_dataframe()}')

In [ ]:
import numpy as np
np.save(os.path.join(project_root, 'Predictions', 'Tide.npy'), finalout.values().flatten())